<a href="https://colab.research.google.com/github/SEOYUNJE/Lung-Image-Analysis/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Factors Influencing Learnable Mechanisms

1. TARGET_SIZE: `224,224` or `384,384`
2. INTERPOLATION: `bilinear`, `nearest`, `bicubic`, `lanczos3`, `lanczos5`.
3. Filters: `16`, `32`, `64`
4. num_res_blocks: `5`, `10`, `15`

In [ ]:
INP_SIZE      = (512, 512) # Input size of the Image Resizer Module (IRM)
TARGET_SIZE   = (224, 224) # Output size of IRM and Input size of the Vision Transformer
INTERPOLATION = "bilinear"

def residual_block(x):
    shortcut = x
    def conv_bn_leaky(inputs, filters, kernel_size, strides):
        x = layers.Conv2D(filters,
                          kernel_size,
                          strides=strides,
                          use_bias=False,
                          padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU()(x)
        return x

    def conv_bn(inputs, filters, kernel_size, strides):
        x = layers.Conv2D(filters,
                          kernel_size,
                          strides,
                          padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        return x

    x = conv_bn_leaky(x, 16, 3, 1)
    x = conv_bn(x, 16, 3, 1)
    x = layers.add([shortcut, x])
    return x


def get_learnable_resizer(filters=16,
                          num_res_blocks=1,
                          interpolation=INTERPOLATION,
                          name=None):

    inputs = layers.Input(shape=[None, None, 3])

    # First, perform naive resizing.
    naive_resize = layers.Resizing(
        *TARGET_SIZE, interpolation=interpolation
    )(inputs)

    # First convolution block without batch normalization.
    x = layers.Conv2D(filters=filters, kernel_size=7, strides=1, padding='same')(inputs)
    x = layers.LeakyReLU(0.2)(x)

    # Second convolution block with batch normalization.
    x = layers.Conv2D(filters=filters, kernel_size=1, strides=1, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.BatchNormalization()(x)

    # Intermediate resizing as a bottleneck.
    bottleneck = layers.Resizing(
        *TARGET_SIZE, interpolation=interpolation
    )(x)

    # Residual passes.
    x = residual_block(bottleneck)
    for i in range(1, num_res_blocks):
        x = residual_block(x)

    # Projection.
    x = layers.Conv2D(
        filters=filters, kernel_size=3, strides=1, padding="same", use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)

    # Skip connection.
    x = layers.Add()([bottleneck, x])

    # Final resized image.
    x = layers.Conv2D(filters=3, kernel_size=7, strides=1, padding="same")(x)
    final_resize = layers.Add()([naive_resize, x])
    return Model(inputs, final_resize, name=name)